# Trabalho 1

## Alunos:
- Diógenes Silva Pedro - 11883476
- Fernando Henrique Paes Generich - 11795342
- Marco Antonio Ribeiro de Toledo - 11796419
- Milena Corrêa da Silva - 11795401

Primeiro vamos importar as bibliotecas necessárias

In [79]:
%pip install glfw numpy PyOpenGL
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

Note: you may need to restart the kernel to use updated packages.


Iniciando janela

In [80]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(600, 600, "Cores", None, None)
glfw.window_hint(glfw.RESIZABLE, glfw.FALSE)
glfw.window_hint(glfw.MAXIMIZED, glfw.FALSE)
glfw.make_context_current(window)

Vertex Code


In [81]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

Fragment Shader

In [82]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

Requisitando um slot da GPU

In [83]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

Associando o código fonte aos slots solicitados

In [ ]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

Compilando com o vertex shader e o fragment shader

In [ ]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

Associando o programa compilado ao programa principal

In [ ]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

Linkagem do programa

In [ ]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

Preparando dados para enviar para a GPU

In [ ]:
# Pontos
verts = [
    (0.0, +0.5),  # vertice 0
    (-0.5, -0.5),  # vertice 1
    (0, -0.25),  # vertice 3
    (+0.5, -0.5),  # vertice 2
]


# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(len(verts), [("position", np.float32, len(verts[0]))])

# preenchendo as coordenadas de cada vértice
vertices['position'] = verts

# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

loc_color = glGetUniformLocation(program, "color")
R = 0.96875
G = 0.96875
B = 0.9453125

Capturando eventos do teclado para e modificando as variáveis para a matriz de transformação

In [ ]:
# exemplo para matriz de translacao
t_x = 0
t_y = 0
r = 0
r_step = 0.05

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, r
    
    # print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)
    # print('-------')
    # if key == 265: t_y += 0.01 #cima
    # if key == 264: t_y -= 0.01 #baixo
    # if key == 263: t_x -= 0.01 #esquerda
    # if key == 262: t_x += 0.01 #direita
    if scancode == 24: r += r_step
    if scancode == 26: r -= r_step
    
glfw.set_key_callback(window,key_event)

In [ ]:
glfw.show_window(window)

Main Loop

In [ ]:
t_y = 0
t_x = 0
r = 0
s = 0.15

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 

    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0.15625, 0.1640625, 0.2109375, 1.0)
    
    
    #Draw Triangle
    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    mat_scale = np.array([  s, 0.0, 0.0, 0.0,
                            0.0, s, 0.0, 0.0,
                            0.0, 0.0, 1.0, 0.0,
                            0.0, 0.0, 0.0, 1.0], np.float32)

    ### apenas para visualizarmos o cubo rotacionando
    cos_r = math.cos(r)
    sin_r = math.sin(r)

    mat_rotation = np.array([   cos_r, -sin_r, 0.0, 0.0, 
                                sin_r,  cos_r, 0.0, 0.0, 
                                0.0,      0.0, 1.0, 0.0, 
                                0.0,      0.0, 0.0, 1.0], np.float32)

    mat_transform = multiplica_matriz(mat_translation, mat_rotation)
    mat_transform = multiplica_matriz(mat_transform, mat_scale)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
    
    glDrawArrays(GL_LINE_LOOP, 0, len(vertices))
    glUniform4f(loc_color, R, G, B, 1.0) ### modificando a cor do objeto!

    glfw.swap_buffers(window)

glfw.terminate()